In [1]:
# Import generic wrappers
from transformers import AutoModelForSequenceClassification, AutoTokenizer 


# Define the model repo
model_name = "pedropei/question-intimacy" 


# Download pytorch model
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


inputs = ["Whats one question you hate being asked?", 
          "How important is aerodynamics in outer space?", 
          "What is the best way to get them to have analsecks?", 
          "What is love when you're at the age of 11?"]


for i in inputs:
    # Transform input tokens 
    t = tokenizer(i, return_tensors="pt")
    
    # Model apply
    output = model(**t)
    
    print(output)

C:\Users\avdhi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\avdhi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:146: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:77.)
  t = torch.tensor([], dtype=storage.dtype, device=storage.untyped().device)
Some weights of the model checkpoint at pedropei/question-intimacy were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializ

SequenceClassifierOutput(loss=None, logits=tensor([[0.3309]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[-0.3655]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[0.4569]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
SequenceClassifierOutput(loss=None, logits=tensor([[0.4109]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [8]:
# language translation

from transformers import pipeline
from tqdm import tqdm

def translation(inputs_fr):
    model_checkpoint = "Helsinki-NLP/opus-mt-fr-en"
    translator = pipeline("translation", model=model_checkpoint)

    inputs = []
    for i_fr in tqdm(inputs_fr):
        inp = translator(i_fr)

        inp = inp[0]['translation_text']

        inputs.append(inp)
    
    return inputs

In [9]:
# quantifying intimacy score

# Import generic wrappers
from transformers import AutoModelForSequenceClassification, AutoTokenizer 
import torch

def calculate_intimacy_score(inputs, label=None):
    # Define the model repo
    model_name = "pedropei/question-intimacy" 


    # Download pytorch model
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, add_prefix_space=True)
    # list of strings should be pretokenized
    # add_prefix_space needed for pretokenized


    # for i in inputs:
    #     # Transform input tokens 
    #     t = tokenizer(i, return_tensors="pt")

    #     labels = torch.tensor([0.517, -0.421, 0.4999, 0.514]).unsqueeze(0)

    #     print(labels)

    #     # Model apply
    #     output = model(**t, labels=labels)

    #     print(output)
    
    t = tokenizer(inputs, return_tensors="pt", is_split_into_words=True)
    if label:
        labels = torch.tensor(label).unsqueeze(0)
        # Model apply
        output = model(**t, labels=labels)
    
    else:
        output = model(**t)

    return output

In [10]:
inputs_fr = ["Quelle est la question que vous détestez qu'on vous pose ?", 
             "Quelle est l'importance de l'aérodynamique dans l'espace ?", 
             "Quelle est la meilleure façon de leur faire avoir des analsecks ?", 
             "Qu'est-ce que l'amour à 11 ans ?"]

labels = [0.517, -0.421, 0.4999, 0.514]

inp_en = translation(inputs_fr)
print(inp_en)

score = calculate_intimacy_score(inp_en, labels)
print(score)

C:\Users\avdhi\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\models\marian\tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  2.10it/s]


["What's the question you hate being asked?", 'What is the importance of aerodynamics in space?', "What's the best way to make them have analsecks?", 'What is 11-year-old love?']


Some weights of the model checkpoint at pedropei/question-intimacy were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SequenceClassifierOutput(loss=tensor(0.1660, grad_fn=<MseLossBackward0>), logits=tensor([[0.3349]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


C:\Users\avdhi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [26]:
import import_ipynb
from load_data import load_data, getLanguageData, k_folds, k_fold_split

k=70

raw_data = load_data("data", "train_copy.csv")['train']

# #English
# en_text, en_label = getLanguageData(raw_data, 'English')
# #French
# fr_text, fr_label = getLanguageData(raw_data, 'French')
# text_folds,label_folds = k_folds(10, {0:en_text,1:fr_text}, {0:en_label,1:fr_label})
# train_text,train_label,validate_text,validate_label = k_fold_split(10, text_folds,label_folds)

#English
print('English')
en_text, en_label = getLanguageData(raw_data, 'English')
en_text_folds,en_label_folds = k_folds(k, [en_text], [en_label])
en_train_text,en_train_label,en_validate_text,en_validate_label = k_fold_split(k, en_text_folds,en_label_folds)

#French
print('\nFrench')
fr_text, fr_label = getLanguageData(raw_data, 'French')
fr_text_folds,fr_label_folds = k_folds(k, [fr_text], [fr_label])
fr_train_text,fr_train_label,fr_validate_text,fr_validate_label = k_fold_split(k, fr_text_folds,fr_label_folds)
# print(text_folds,label_folds)

for fold in range(k):
    text = en_validate_text[fold]
    print("Translating French to English")
#     train_text.extend(translation(fr_validate_text[fold]))
    print('text:',len(text))

    label = en_validate_label[fold]
#     train_label.extend(fr_validate_label[fold])
    print('Train_label:',len(label))

    print("Predicting Intimacy Scores")
    score = calculate_intimacy_score(text, label)
    print(score)
#     break

Using custom data configuration data-1ba08ea01a127387
Found cached dataset csv (C:/Users/avdhi/.cache/huggingface/datasets/csv/data-1ba08ea01a127387/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 260.94it/s]


English
Folding...
Fold 1 : 22 data
Fold 2 : 22 data
Fold 3 : 22 data
Fold 4 : 22 data
Fold 5 : 22 data
Fold 6 : 22 data
Fold 7 : 22 data
Fold 8 : 22 data
Fold 9 : 22 data
Fold 10 : 22 data
Fold 11 : 22 data
Fold 12 : 22 data
Fold 13 : 22 data
Fold 14 : 22 data
Fold 15 : 22 data
Fold 16 : 22 data
Fold 17 : 22 data
Fold 18 : 22 data
Fold 19 : 22 data
Fold 20 : 22 data
Fold 21 : 22 data
Fold 22 : 22 data
Fold 23 : 22 data
Fold 24 : 22 data
Fold 25 : 22 data
Fold 26 : 22 data
Fold 27 : 22 data
Fold 28 : 22 data
Fold 29 : 22 data
Fold 30 : 22 data
Fold 31 : 22 data
Fold 32 : 22 data
Fold 33 : 22 data
Fold 34 : 22 data
Fold 35 : 22 data
Fold 36 : 22 data
Fold 37 : 22 data
Fold 38 : 22 data
Fold 39 : 22 data
Fold 40 : 22 data
Fold 41 : 22 data
Fold 42 : 22 data
Fold 43 : 22 data
Fold 44 : 22 data
Fold 45 : 22 data
Fold 46 : 22 data
Fold 47 : 22 data
Fold 48 : 22 data
Fold 49 : 22 data
Fold 50 : 22 data
Fold 51 : 22 data
Fold 52 : 22 data
Fold 53 : 22 data
Fold 54 : 22 data
Fold 55 : 22 data


Some weights of the model checkpoint at pedropei/question-intimacy were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


SequenceClassifierOutput(loss=tensor(0.1881, grad_fn=<MseLossBackward0>), logits=tensor([[-0.0935]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


C:\Users\avdhi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([22])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
